Smith Normalization
===
This code make Smith Normale Form from an integer matrix.

In [1]:
import numpy as np



In [2]:
# 整数行列上の基本行列
def Eij(i,j,n):
    E = np.eye(n)
    E[i,i] = 0
    E[j,j] = 0
    E[i,j] = 1
    E[j,i] = 1
    return E

def Ei(i,n):
    E = np.eye(n)
    E[i,i] = -1
    return E

def Ec(i,j,c,n):
    E = np.eye(n)
    E[i,j] = c
    return E    

In [3]:
# A[k:,k:]上の0以外の最小の絶対値をA[k,k]に移動する変換
def moveMN(A,k):
    tmp_A = A[k:,k:]
    a = np.abs(tmp_A[tmp_A != 0]).min()
    i = np.where(np.abs(tmp_A) == a)[0][0] + k
    j = np.where(np.abs(tmp_A) == a)[1][0]+ k
    P = Eij(k,j,A.shape[1])
    invQ = Eij(i,k,A.shape[0])
    B = invQ.dot(A).dot(P)
    if B[k,k]<0:
        Pi =Ei(k,A.shape[1])
        B = B.dot(Pi)
        P = P.dot(Pi)
    return invQ.astype(int),B.astype(int),P.astype(int)



In [4]:
#A[k,k]を使って，A[k+1:,k]を0に整える変換．(整いきれなかったら要素はA[k,k]より小さくなる)
def rowR(A,k):
    B = A.copy()
    invQ = np.eye(A.shape[0])
    P = np.eye(A.shape[1])
    for i in range(k+1,A.shape[0]):
        q = A[i,k]//A[k,k]
        #残渣
        r = A[i,k]%A[k,k]
        invQi = Ec(i,k,-q,A.shape[0])
        B = invQi.dot(B)
        invQ = invQi.dot(invQ)
    return invQ.astype(int),B.astype(int),P.astype(int)

In [5]:
#A[k,k]を使って，A[k,k+1]を0に整える変換．(整いきれなかったら要素はA[k,k]より小さくなる)
def colR(A,k):
    B = A.copy()
    invQ = np.eye(A.shape[0])
    P = np.eye(A.shape[1])
    for i in range(k+1,A.shape[1]):
        q = A[k,i]//A[k,k]
        #残渣
        r = A[k,i]%A[k,k]
        Pi = Ec(k,i,-q,A.shape[1])
        B = B.dot(Pi)
        P = P.dot(Pi)
    return invQ.astype(int),B.astype(int),P.astype(int)

In [6]:
# A[k+1:,k+1:]においてA[k,k]で割り切れない要素A[i,j]をA[k,k]の残差に変換する変換
def remR(A,k):
    invQ = np.eye(A.shape[0])
    P = np.eye(A.shape[1])
    #  Find i,j
    i = np.where(A[k+1:,k+1:]%A[k,k] !=0)[0][0] +k+1
    j = np.where(A[k+1:,k+1:]%A[k,k] !=0)[1][0] +k+1
    q = A[i,j]//A[k,k]
    r = A[i,j]%A[k,k]
    invQi = Ec(i,k,q,A.shape[0])
    Pi = Ec(k,j,-1,A.shape[1])
    B = invQi.dot(A).dot(Pi)
    P = P.dot(Pi)
    invQ = invQi.dot(invQ)
    return invQ.astype(int),B.astype(int),P.astype(int)

In [7]:
def Smith_Normalization(A):
    invQ = np.eye(A.shape[0])
    P = np.eye(A.shape[1])
    A0 = A.copy()
    # limit of optimization
    N = 1000
    for k in range(min(A0.shape)):
        # If A0[k:,k:] is zero matrix, then stop calculation
        if np.sum(np.abs(A0[k:,k:]))==0:
            break
        for i in range(N):
            if i == N-1 : 
                print("Error: Time Out")
            # minimize A[k,k]
            invQi,A1,Pi = moveMN(A0,k)
            invQ = invQi.dot(invQ)
            P = P.dot(Pi)
            # make zero row A[k+1:,k]
            invQi,A2,Pi = rowR(A1,k)
            invQ = invQi.dot(invQ)
            P = P.dot(Pi)
            # if row A2[k+1:,k] is zero vector ?
            if np.abs(A2[k+1:,k]).sum() ==0:
                # make zero col A[k,k+1:]
                invQi,A3,Pi = colR(A2,k)
                invQ = invQi.dot(invQ)
                P = P.dot(Pi)
                # if col A3[k+1:,k] is zero vector ?
                if np.abs(A3[k,k+1:]).sum() ==0:
                    # A[k,k]|A[k+1:,k+1:]?
                    if np.sum(A3[k+1:,k+1:]%A3[k,k]) == 0:                    
                        A0 = A3.copy()            
                        break;
                    else:
                        # reduce A[k+1:,k+1:]
                        invQi,A0,Pi = remR(A3,k)
                        invQ = invQi.dot(invQ)
                        P = P.dot(Pi)
                else:
                    A0 = A3.copy()            
            else:
                A0 = A2.copy()

    B = A0.copy().astype(int)
    P = P.astype(int)
    invQ = invQ.astype(int)
    return invQ,B,P




In [8]:
A = np.random.randint(0,10,(3,4))
A

array([[7, 3, 2, 1],
       [7, 6, 7, 7],
       [4, 8, 2, 0]])

# Function check

In [9]:
A1 = moveMN(A,0)[1]
A1

array([[1, 3, 2, 7],
       [7, 6, 7, 7],
       [0, 8, 2, 4]])

In [10]:
A2 = rowR(A1,0)[1]
A2

array([[  1,   3,   2,   7],
       [  0, -15,  -7, -42],
       [  0,   8,   2,   4]])

In [11]:
A3 = colR(A2,0)[1]
A3

array([[  1,   0,   0,   0],
       [  0, -15,  -7, -42],
       [  0,   8,   2,   4]])

In [12]:
tmpA = np.array([[2,0,0,0],[0,2,3,4],[0,2,2,6]])
tmpA

array([[2, 0, 0, 0],
       [0, 2, 3, 4],
       [0, 2, 2, 6]])

In [13]:
A4 = remR(tmpA,0)[1]
A4

array([[ 2,  0, -2,  0],
       [ 2,  2,  1,  4],
       [ 0,  2,  2,  6]])

## Main result

In [14]:
invQ,B,P = Smith_Normalization(A)
print(invQ)
print(B)
print(P)

[[ 1  0  0]
 [-7  1  4]
 [14 -2 -7]]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 2 0]]
[[   0    0   -6   13]
 [   0    0  -13   28]
 [   0    1   65 -138]
 [   1   -2  -49  101]]


In [15]:
print(invQ.dot(A).dot(P))

[[1 0 0 0]
 [0 1 0 0]
 [0 0 2 0]]


In [16]:
invP = np.linalg.inv(P)
# float to int
invP = np.round(invP).astype(int)
print(invP)
print(P.dot(invP))

[[  7   3   2   1]
 [-26  17   1   0]
 [ 28 -13   0   0]
 [ 13  -6   0   0]]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [17]:
Q = np.linalg.inv(invQ)
# float to int
Q = np.round(Q).astype(int)
print(Q)
print(Q.dot(invQ))

[[ 1  0  0]
 [ 7 -7 -4]
 [ 0  2  1]]
[[1 0 0]
 [0 1 0]
 [0 0 1]]
